In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def unpack_pickle(obj_name):
    file_path = "{0}.pkl".format(obj_name)
    n_bytes = 2**31
    max_bytes = 2**31 - 1
    data = bytearray(n_bytes)
    
    ## read
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    loaded_data = pickle.loads(bytes_in)
    return loaded_data

In [3]:
X = unpack_pickle('X')

In [4]:
y = unpack_pickle('y')

In [5]:
X.shape, y.shape

((7730792, 73), (7730792,))

In [6]:
subm = pd.read_csv('../sample_submission.csv')

In [7]:
subm.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.5
1,00000176ac758d54827acd545b6315a5,0.5
2,0000019dcefc128c2d4387c1273dae1d,0.5
3,0000055553dc51b1295785415f1a224d,0.5
4,00000574cefffeca83ec8adf9285b2bf,0.5


In [8]:
test = unpack_pickle('test')

In [9]:
test.shape, subm.shape

((7853253, 73), (7853253, 2))

In [10]:
X.columns.dtype == test.columns.dtype

True

In [11]:
test.select_dtypes(include='category').columns.tolist()

['Processor',
 'OsPlatformSubRelease',
 'SkuEdition',
 'Census_PrimaryDiskTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSArchitecture',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'EngineVersion_1',
 'EngineVersion_2',
 'AppVersion_1',
 'AppVersion_2',
 'AvSigVersion_1',
 'Census_OSVersion_1']

In [12]:
dtypes = {}
for i in X.columns:
    dtypes[i] = str(X[i].dtype)

In [13]:
dtypes

{'AVProductStatesIdentifier': 'float32',
 'AVProductsInstalled': 'float16',
 'AppVersion': 'float64',
 'AppVersion_1': 'float64',
 'AppVersion_2': 'float64',
 'AppVersion_3': 'float64',
 'AvSigVersion': 'float64',
 'AvSigVersion_1': 'float64',
 'AvSigVersion_2': 'float64',
 'AvSigVersion_3': 'float64',
 'Census_ActivationChannel': 'float64',
 'Census_ChassisTypeName': 'float64',
 'Census_FirmwareManufacturerIdentifier': 'float16',
 'Census_FirmwareVersionIdentifier': 'float32',
 'Census_FlightRing': 'float64',
 'Census_GenuineStateName': 'float64',
 'Census_HasOpticalDiskDrive': 'int8',
 'Census_InternalBatteryNumberOfCharges': 'float32',
 'Census_InternalBatteryType': 'float64',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches': 'float16',
 'Census_InternalPrimaryDisplayResolutionHorizontal': 'float16',
 'Census_InternalPrimaryDisplayResolutionVertical': 'float16',
 'Census_IsAlwaysOnAlwaysConnectedCapable': 'float16',
 'Census_IsSecureBootEnabled': 'int8',
 'Census_IsTouchEnabled'

In [14]:
for i in dtypes:
    test[i] = test[i].astype(dtypes[i])

In [15]:
test.select_dtypes(include='category').columns.tolist()

[]

In [16]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc
gc.enable()

In [17]:
params = {
    'boosting_type': 'gbdt',
    'class_weight': None,
    'colsample_bytree': 0.9,
    'learning_rate': 0.1,
    'max_depth': 40,
    'metric': 'auc',
    'min_child_samples': 20,
    'min_child_weight': 0.001,
    'min_split_gain': 0.0,
    'n_estimators': 4000,
    'n_jobs': 4,
    'num_leaves': 31, 
    'objective': 'binary',
    'random_state': None,
    'reg_alpha': 0.1,
    'reg_lambda': 5,
    'silent': False,
    'subsample': 0.7,
    'subsample_for_bin': 200000,
    'subsample_freq': 0
}

In [18]:
sub=subm['MachineIdentifier'].to_frame()
sub['HasDetections']=0

In [19]:
sub.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0
1,00000176ac758d54827acd545b6315a5,0
2,0000019dcefc128c2d4387c1273dae1d,0
3,0000055553dc51b1295785415f1a224d,0
4,00000574cefffeca83ec8adf9285b2bf,0


In [20]:
kfold = 5

In [21]:
X = X.values
y = y.values

In [22]:
test = test.values

In [23]:
gc.collect()

21

In [24]:
skf = StratifiedKFold(n_splits=kfold, random_state=2200)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    dset = lgb.Dataset(X_train, label=y_train)
    dset_eval = lgb.Dataset(X_eval, label=y_eval)
    lgb_model = lgb.train(params, dset, 1000, dset_eval,
                          verbose_eval=100, early_stopping_rounds=100)
    sub['HasDetections'] += lgb_model.predict(test, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    
sub.to_csv('third_submit.csv', index=False, float_format='%.5f') 
gc.collect()
sub.head()

 lgb kfold: 1  of  5 : 


/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:661: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.717049
[200]	valid_0's auc: 0.722814
[300]	valid_0's auc: 0.725699
[400]	valid_0's auc: 0.727464
[500]	valid_0's auc: 0.728853
[600]	valid_0's auc: 0.729813
[700]	valid_0's auc: 0.730646
[800]	valid_0's auc: 0.731569
[900]	valid_0's auc: 0.732149
[1000]	valid_0's auc: 0.732718
[1100]	valid_0's auc: 0.733237
[1200]	valid_0's auc: 0.733759
[1300]	valid_0's auc: 0.734165
[1400]	valid_0's auc: 0.734485
[1500]	valid_0's auc: 0.734767
[1600]	valid_0's auc: 0.735117
[1700]	valid_0's auc: 0.735329
[1800]	valid_0's auc: 0.73555
[1900]	valid_0's auc: 0.735844
[2000]	valid_0's auc: 0.73612
[2100]	valid_0's auc: 0.736233
[2200]	valid_0's auc: 0.736379
[2300]	valid_0's auc: 0.736536
[2400]	valid_0's auc: 0.736677
[2500]	valid_0's auc: 0.7368
[2600]	valid_0's auc: 0.73689
[2700]	valid_0's auc: 0.737019
[2800]	valid_0's auc: 0.737152
[2900]	valid_0's auc: 0.737261
[3000]	valid_0's auc: 0.737338
[3100]	valid_0's auc:

/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:661: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.716448
[200]	valid_0's auc: 0.722314
[300]	valid_0's auc: 0.725298
[400]	valid_0's auc: 0.727172
[500]	valid_0's auc: 0.728478
[600]	valid_0's auc: 0.729645
[700]	valid_0's auc: 0.730482
[800]	valid_0's auc: 0.731288
[900]	valid_0's auc: 0.73193
[1000]	valid_0's auc: 0.7325
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.7325
 lgb kfold: 3  of  5 : 
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.716999
[200]	valid_0's auc: 0.722953
[300]	valid_0's auc: 0.725786
[400]	valid_0's auc: 0.727729
[500]	valid_0's auc: 0.729153
[600]	valid_0's auc: 0.730217
[700]	valid_0's auc: 0.731056
[800]	valid_0's auc: 0.731875
[900]	valid_0's auc: 0.732468
[1000]	valid_0's auc: 0.733058
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.733058
 lgb kfold: 4  of  5 : 
Training until validation scores don't improve for 100 rounds.
[100]	va

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.584937
1,00000176ac758d54827acd545b6315a5,0.693437
2,0000019dcefc128c2d4387c1273dae1d,0.585016
3,0000055553dc51b1295785415f1a224d,0.493170
4,00000574cefffeca83ec8adf9285b2bf,0.577813
